In [ ]:
# Напишем оптимальный запрос, который даст информацию о количестве очень усердных студентов.
# Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.
SELECT COUNT(*)
FROM (
    SELECT st_id
    FROM peas
    WHERE correct = true
    GROUP BY st_id
    HAVING COUNT(st_id) >= 20
) AS subquery

# по итогам запроса мы получили 136 активных студентов

In [ ]:
# В одном запросе выгрузим следующую информацию о группах пользователей:
#ARPU 
#ARPAU 
#CR в покупку 
#СR активного пользователя в покупку 
#CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

WITH active_and_math_flags AS (
    SELECT 
        s.st_id, 
        s.test_grp,
        SUM(CASE WHEN p.correct = 'true' THEN 1 ELSE 0 END) AS total_correct,
        SUM(CASE WHEN p.subject = 'Math' AND p.correct = 'true' THEN 1 ELSE 0 END) AS math_correct
    FROM studs s
    LEFT JOIN peas p ON s.st_id = p.st_id
    GROUP BY s.st_id, s.test_grp
),
user_stats AS (
    SELECT 
        s.test_grp,
        COUNT(DISTINCT s.st_id) AS all_users,
        COUNT(DISTINCT CASE WHEN a.total_correct > 10 THEN s.st_id END) AS active_users,
        COUNT(DISTINCT CASE WHEN a.math_correct >= 2 THEN s.st_id END) AS active_in_math
    FROM studs s
    LEFT JOIN active_and_math_flags a ON s.st_id = a.st_id
    GROUP BY s.test_grp
),
payment_stats AS (
    SELECT 
        s.test_grp,
        SUM(f.money) AS revenue,
        COUNT(DISTINCT f.st_id) AS paying_users,
        SUM(CASE WHEN a.total_correct > 10 THEN f.money ELSE 0 END) AS revenue_from_active,
        COUNT(DISTINCT CASE WHEN a.total_correct > 10 THEN f.st_id END) AS active_paying_users,
        COUNT(DISTINCT CASE WHEN a.math_correct >= 2 THEN f.st_id END) AS active_paying_in_math
    FROM studs s
    LEFT JOIN final_project_check f ON s.st_id = f.st_id
    LEFT JOIN active_and_math_flags a ON s.st_id = a.st_id
    GROUP BY s.test_grp
)
SELECT 
    ps.test_grp,
    ps.revenue / NULLIF(us.all_users, 0) AS ARPU,
    ps.revenue_from_active / NULLIF(us.active_users, 0) AS ARPAU,
    (ps.paying_users::float / NULLIF(us.all_users, 0)) * 100 AS CR,
    (ps.active_paying_users::float / NULLIF(us.active_users, 0)) * 100 AS CR_from_active,
    (ps.active_paying_in_math::float / NULLIF(us.active_in_math, 0)) * 100 AS CR_math
FROM 
    payment_stats ps
JOIN 
    user_stats us ON ps.test_grp = us.test_grp;
